In [ ]:
import numpy as np 
import pandas as pd 
import pandas as pd
from geopy.geocoders import Nominatim
import time

data = pd.read_csv("df_UK_lat_lon.csv")

In [5]:
data.columns

Index(['id', 'RLOIid', 'catchmentName', 'dateOpened', 'datumOffset', 'easting',
       'label', 'lat', 'long', 'northing', 'notation', 'riverName',
       'stageScale', 'stationReference', 'status', 'town', 'wiskiID',
       'measures'],
      dtype='object')

In [ ]:

df = data

# Initialize the geolocator
geolocator = Nominatim(user_agent="town_filler")

# Step 1: Identify unique (lat, long) pairs where town is missing
missing_towns = df[df['town'].isna()][['lat', 'long']].drop_duplicates()

# Step 2: Dictionary to store geocoded towns
town_dict = {}

# Step 3: Geocode each unique coordinate pair with a delay
for index, row in missing_towns.iterrows():
    lat = row['lat']
    long = row['long']
    try:
        # Perform reverse geocoding
        location = geolocator.reverse((lat, long))
        if location and 'address' in location.raw:
            address = location.raw['address']
            # Extract town, city, or village (whichever is available)
            town = address.get('town') or address.get('city') or address.get('village')
            town_dict[(lat, long)] = town
    except Exception as e:
        print(f"Error geocoding {lat}, {long}: {e}")
    # Manual delay to respect service limits (1 request per second)
    time.sleep(1)

# Step 4: Function to fill missing towns
def fill_town(row):
    if pd.isna(row['town']) or row['town'] == '':
        key = (row['lat'], row['long'])
        # Return the geocoded town if available, else keep original
        return town_dict.get(key, row['town'])
    return row['town']

# Step 5: Apply the function to the DataFrame
df['town'] = df.apply(fill_town, axis=1)

# Display the updated DataFrame
print(df)

Error geocoding nan, nan: Must be a coordinate pair or Point


In [ ]:
import requests 
import json
import numpy as np
import pandas as pd

data_request = requests.get('https://environment.data.gov.uk/flood-monitoring/id/stations')
read = data_request.json()
temp_lis = []
for i in read['items']:
    temp_lis.append(i)
columns = []
for j in i.keys():
    j = str(j)
    columns.append(j)

In [1]:
import json

with open('uk.geojson', 'r') as f:
    geojson_data = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'uk.geojson'

In [ ]:
data = temp_lis

flattened_data = []
for station in data:
    station_dict = {
        'id': station.get('@id'),
        'RLOIid': station.get('RLOIid'),
        'catchmentName': station.get('catchmentName'),
        'dateOpened': station.get('dateOpened'),
        'datumOffset': station.get('datumOffset'),
        'easting': station.get('easting'),
        'label': station.get('label'),
        'lat': station.get('lat'),
        'long': station.get('long'),
        'northing': station.get('northing'),
        'notation': station.get('notation'),
        'riverName': station.get('riverName'),
        'stageScale': station.get('stageScale'),
        'stationReference': station.get('stationReference'),
        'status': station.get('status'),
        'town': station.get('town'),
        'wiskiID': station.get('wiskiID'),
        'measures': json.dumps(station.get('measures', []))  # Convert measures list to string
    }
    flattened_data.append(station_dict)

df = pd.DataFrame(flattened_data)

print(df.head())

                                                  id RLOIid  \
0  http://environment.data.gov.uk/flood-monitorin...   7041   
1  http://environment.data.gov.uk/flood-monitorin...   6022   
2  http://environment.data.gov.uk/flood-monitorin...   3072   
3  http://environment.data.gov.uk/flood-monitorin...   6177   
4  http://environment.data.gov.uk/flood-monitorin...   2032   

                             catchmentName  dateOpened  datumOffset easting  \
0                                Cotswolds  1994-01-01          NaN  417990   
1                                  Welland  1992-01-01          2.0  528000   
2  Parrett, Brue and West Somerset Streams  1997-01-01         10.0  344383   
3                   Upper and Bedford Ouse  1996-10-01          6.3  529500   
4             Worcestershire Middle Severn  1976-04-06          NaN  386120   

             label        lat      long northing notation         riverName  \
0  Bourton Dickler  51.874767 -1.740083   219610   1029TH      Rive

In [ ]:
df.nunique()

TypeError: unhashable type: 'list'

In [ ]:
df.to_csv('df_UK_lat_lon.csv', index=False)

In [ ]:
data = pd.read_csv("df_UK_lat_lon.csv")

In [ ]:
data.columns

Index(['id', 'RLOIid', 'catchmentName', 'dateOpened', 'datumOffset', 'easting',
       'label', 'lat', 'long', 'northing', 'notation', 'riverName',
       'stageScale', 'stationReference', 'status', 'town', 'wiskiID',
       'measures'],
      dtype='object')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5392 entries, 0 to 5391
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                5392 non-null   object 
 1   RLOIid            2526 non-null   object 
 2   catchmentName     2526 non-null   object 
 3   dateOpened        2526 non-null   object 
 4   datumOffset       294 non-null    float64
 5   easting           4758 non-null   object 
 6   label             5392 non-null   object 
 7   lat               4758 non-null   object 
 8   long              4758 non-null   object 
 9   northing          4758 non-null   object 
 10  notation          5391 non-null   object 
 11  riverName         2526 non-null   object 
 12  stageScale        2526 non-null   object 
 13  stationReference  5391 non-null   object 
 14  status            2526 non-null   object 
 15  town              2367 non-null   object 
 16  wiskiID           2511 non-null   object 
